In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
lin = nn.Linear(5, 3)
print (lin)
data = autograd.Variable(torch.randn(2, 5))
print (data)
print (lin(data))

In [ ]:
data = autograd.Variable(torch.randn(2, 2))
print (F.relu(data))
print (F.softmax(data).sum())

data = autograd.Variable(torch.randn(5))
print (F.softmax(data))
print (F.log_softmax(data))

In [ ]:
# log Softmax(Ax + b)
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

print (data[0][0])
# print (data + test_data)

In [ ]:
word_to_index = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)
print (word_to_index)

VOCAB_SIZE = len(word_to_index)
NUM_LABELS = 2

In [ ]:
class BOWClassifier(nn.Module):
    def __init__(self, num_labels, vocab_size):
        super(BOWClassifier, self).__init__()
        self.linear = nn.Linear(vocab_size, num_labels)
    
    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec))
    
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [ ]:
model = BOWClassifier(NUM_LABELS, VOCAB_SIZE)
for param in model.parameters():
    print (param)    

In [ ]:
sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_index)
print (bow_vector.shape)
log_probs = model(autograd.Variable(bow_vector))
print (log_probs)

In [ ]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}
for instance, label in test_data:
    bow_vector = autograd.Variable(make_bow_vector(instance, word_to_index))
    log_probs = model(bow_vector)
    print (log_probs)

print (next(model.parameters())[:, word_to_index["creo"]])

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# print (next(model.parameters()))
for epoch in range(500):
    for instance, label in data:
        model.zero_grad()
        bow_vector = autograd.Variable(make_bow_vector(instance, word_to_index))
        label_vector = autograd.Variable(make_target(label, label_to_ix))
        # print (label_vector)
        # print (bow_vector)
        
        log_prob = model(bow_vector)
        loss = loss_function(log_prob, label_vector)
        loss.backward()
        optimizer.step()
# print (next(model.parameters()))

for instance, label in test_data:
    bow_vector = autograd.Variable(make_bow_vector(instance, word_to_index))
    log_probs = model(bow_vector)
    print (log_probs)

print (next(model.parameters())[:, word_to_index["creo"]])        